# Configurando o ambiente

Usei a integtração SQL com o Jupyter Notebook para rodar as queries do exercício de forma interativa e mais fácil de acompanhar.


In [8]:
%load_ext sql

There's a new jupysql version available (0.10.2), you're running 0.10.1. To upgrade: pip install jupysql --upgrade


In [9]:
%sql --section db-aula-06

# Exercício


1. Obtenha o nome de cada aluno e o nome do curso a que cada um pertence.


In [14]:
%%sql

select nome_aluno,  nome_curso
from aluno 
left join curso on aluno.cod_curso = curso.cod_curso

Running query in 'db-aula-06'

5 rows affected.

nome_aluno,nome_curso
Claudia,Ciencia da Computacao
Andrea,Ciencia da Computacao
Regiane,Ciencia da Computacao
Rodrigo,Administracao de Empresas
Renata,Administracao de Empresas


2. Obtenha a maior idade, a menor idade e a idade média dos alunos do curso de 'Ciencia da Computacao'.


In [16]:
%%sql

select 
    max(idade) as idade_máxima,
    min(idade) as idade_mínima,
    avg(idade) as idade_média
from aluno 
left join curso on aluno.cod_curso = curso.cod_curso
where nome_curso = 'Ciencia da Computacao'

Running query in 'db-aula-06'

1 rows affected.

idade_máxima,idade_mínima,idade_média
24,20,22.0000000000000000


3. Obtenha o nome de cada curso (que tenha aluno) e, para cada curso, a média de idade e a quantidade de alunos.


In [19]:
%%sql

select 
    nome_curso,
    avg(idade) as idade_média,
    count(matricula) as quantidade_de_alunos
from curso 
inner join aluno on aluno.cod_curso = curso.cod_curso
group by nome_curso

Running query in 'db-aula-06'

2 rows affected.

nome_curso,idade_média,quantidade_de_alunos
Ciencia da Computacao,22.0000000000000000,3
Administracao de Empresas,21.0000000000000000,2


4. Obtenha o nome de cada aluno da universidade (mesmo aqueles que ainda não cursaram disciplina) e a quantidade de disciplinas que cada um já cursou. Retorne em ordem crescente da quantidade de disciplinas e, se houver empate na quantidade, em ordem crescente de nome do aluno.


In [21]:
%%sql

select 
    nome_aluno,
    count(cod_disciplina) as quantidade_de_disciplinas
from aluno
left join aluno_disc on aluno.matricula = aluno_disc.matricula
group by nome_aluno
order by quantidade_de_disciplinas asc

Running query in 'db-aula-06'

5 rows affected.

nome_aluno,quantidade_de_disciplinas
Andrea,0
Regiane,0
Renata,0
Claudia,2
Rodrigo,3


5. Obtenha o nome das disciplinas que a aluna 'Claudia' cursou e a nota que ela tirou em cada uma delas.


In [24]:
%%sql

select 
    nome_aluno,
    nome_disciplina,
    nota
from aluno_disc
left join aluno on aluno.matricula = aluno_disc.matricula
left join disciplina on disciplina.cod_disciplina = aluno_disc.cod_disciplina
where nome_aluno = 'Claudia'
order by nota desc

Running query in 'db-aula-06'

2 rows affected.

nome_aluno,nome_disciplina,nota
Claudia,Banco de Dados I,8
Claudia,Engenharia de Software I,7


6. Obtenha o nome de cada disciplina e o nome do instituto a que cada uma pertence.


In [27]:
%%sql

select
    nome_disciplina,
    nome_instituto
from disciplina as disc
left join curso as c on c.cod_curso = disc.cod_curso
left join instituto as inst on inst.cod_instituto = c.cod_instituto

Running query in 'db-aula-06'

5 rows affected.

nome_disciplina,nome_instituto
Banco de Dados I,Ciencias Exatas
Engenharia de Software I,Ciencias Exatas
Banco de Dados II,Ciencias Exatas
Interface Homem-Maquina,Ciencias Exatas
Engenharia de Software II,Ciencias Exatas


7. Obtenha o nome de todos os professores da universidade (mesmo aqueles que ainda não ministraram disciplina) e os nomes das disciplinas que eles ministram. Retorne em ordem crescente do nome do professor.


In [30]:
%%sql

-- Estou usando Postgres como bando de dados
select
    nome_professor,
    STRING_AGG(nome_disciplina, ', ') as disciplinas
from professor as prof
left join prof_disc as pd on pd.cod_professor = prof.cod_professor
left join disciplina as disc on disc.cod_disciplina = pd.cod_disciplina
group by nome_professor
order by nome_professor asc

Running query in 'db-aula-06'

4 rows affected.

nome_professor,disciplinas
Ana,None
Camila,"Banco de Dados I, Banco de Dados II"
Joao,"Engenharia de Software I, Engenharia de Software II"
Pedro,Interface Homem-Maquina


8. Obtenha o nome dos alunos (de qualquer curso) que têm idade superior à média de idade dos alunos do curso de 'Administracao de Empresas'.


In [41]:
%%sql

select nome_aluno
from aluno
where idade > (select avg(idade)
                 from aluno as al
                 left join curso as c on c.cod_curso = al.cod_curso
                 where nome_curso = 'Administracao de Empresas')

Running query in 'db-aula-06'

3 rows affected.

nome_aluno
Andrea
Regiane
Renata


9. Obtenha o nome de cada professor (que orienta aluno) e a quantidade de alunos que cada um orienta, mas somente aqueles que orientam mais alunos que o professor ‘Joao’.


In [56]:
%%sql

with alunos_por_professor as (
    select nome_professor, count(matricula) as numero_de_alunos
    from aluno al
    left join professor prof on prof.cod_professor = al.cod_professor_orientador
    group by nome_professor
)

select nome_professor, numero_de_alunos
from alunos_por_professor
where numero_de_alunos > (select numero_de_alunos from alunos_por_professor where nome_professor = 'Joao')

Running query in 'db-aula-06'

2 rows affected.

nome_professor,numero_de_alunos
Pedro,2
Camila,2
